In [1]:
import nltk
nltk.download
nltk.download('punkt')
from nltk.corpus import stopwords

stopwords_en = stopwords.words ("english")

def preprocessing (raw):
    word_list = nltk.word_tokenize (raw)
    text = [w.lower() for w in word_list if w not in stopwords_en]
    return text

f1 = open('summary1.txt', 'r')
text1 = preprocessing (f1.read())

f2 = open ('summary2.txt', 'r')
text2 = preprocessing (f2.read())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from nltk.probability import FreqDist

word_set = set(text1).union(set(text2))

freqd_text1 = FreqDist(text1)
text1_count_dict = dict.fromkeys(word_set,0)
for word in text1:
    text1_count_dict[word] = freqd_text1[word]

freqd_text2 = FreqDist(text2)
text2_count_dict = dict.fromkeys(word_set,0)
for word in text2:
    text2_count_dict[word]=freqd_text2[word]
print(text1_count_dict)
print(text2_count_dict)

{'covering': 1, 'one': 3, 'crying': 0, 'on': 2, 'fell': 1, 'breaks': 0, 'would': 1, 'land': 1, 'stones': 2, 'inquiry': 1, 'from': 1, 'playwright': 1, 'bird': 15, 'two': 1, 'precious': 3, 'near': 1, 'body': 2, 'form': 0, 'prince': 9, 'happy': 5, 'refusal': 1, 'condition': 1, 'weep': 1, 'weeping': 1, 'angels': 1, 'furnace': 1, 'garbage': 1, 'become': 2, 'merry': 1, 'weather': 2, 'high': 0, 'survival': 1, 'seamstress': 1, 'broke': 1, 'death': 2, 'told': 1, 'among': 1, 'girl': 1, 'young': 1, 'swallow': 4, 'thrown': 1, 'hilt': 1, 'kiss': 1, 'messenger': 1, 'eyes': 2, 'convinced': 1, 'broken': 1, 'got': 1, 'nothing': 0, 'strange': 1, 'angel': 0, 'sight': 1, 'tall': 1, 'tells': 0, 'distribute': 1, 'goodbye': 1, 'delivered': 1, 'dead': 1, 'the': 8, 'third': 1, 'misery': 2, 'loved': 1, 'gemstones': 0, 'unable': 1, 'inform': 1, 'towards': 1, 'state': 0, 'covered': 1, 'bid': 1, 'lifetime': 0, 'beautiful': 0, 'never': 0, 'shelter': 1, 'becoming': 1, 'became': 2, 'middle': 1, 'sapphire': 3, 'pained

In [3]:
freqd_text1 = FreqDist (text1)
text1_length = len(text1)
text1_tf_dict = dict.fromkeys(word_set, 0)
for word in text1:
    text1_tf_dict[word] = freqd_text1[word]/text1_length

freqd_text2 = FreqDist(text2)
text2_length = len(text2)
text2_tf_dict = dict.fromkeys(word_set, 0)
for word in text2:
    text2_tf_dict[word] = freqd_text2[word]/text2_length

In [4]:
text12_idf_dict = dict.fromkeys (word_set,0)
text12_length = 2 # 2 documents
for word in text12_idf_dict.keys () :
    if word in text1:

        text12_idf_dict [word] += 1
    
    if word in text2:
        text12_idf_dict [word] += 1

import math
for word, val in text12_idf_dict.items():
    text12_idf_dict[word] = 1 + math.log(text12_length/ (float(val) ) )

In [5]:
# TF-IDF Calculations
text1_tfidf_dict = dict.fromkeys (word_set, 0)
for word in text1:
    text1_tfidf_dict[word] = (text1_tf_dict[word]) * (text12_idf_dict[word])

text2_tfidf_dict = dict.fromkeys(word_set, 0)
for word in text2:
    text2_tfidf_dict[word] = (text2_tf_dict[word]) * (text12_idf_dict[word])

In [6]:
import gensim.models.doc2vec
from gensim.models.doc2vec import TaggedDocument

taggeddocs = []
doc1 = TaggedDocument(words=text1, tags=[u'NEWS_1'])
taggeddocs.append(doc1)
doc2 = TaggedDocument(words=text2, tags=[u'NEWS_2'])
taggeddocs.append(doc2)

# build the model
model = gensim.models.Doc2Vec(taggeddocs, dm=0, alpha=0.025, size=20, min_alpha=0.025, min_count=0)

# training
for epoch in range (80):
    if epoch % 20 == 0:
        print ('Now training epoch %s' % epoch)
    
    model.train(taggeddocs,total_examples=model.corpus_count,epochs=model.epochs)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no decay

D:\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\anaconda\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60


In [7]:
# Compute Cosine distance
v1 = list(text1_tfidf_dict.values())
v2 = list(text2_tfidf_dict.values ())
similarity = 1 - nltk.cluster.cosine_distance(v1,v2)
print ("Similarity Index:{:4.2f} %".format(similarity*100))

Similarity Index:60.84 %


In [8]:
similarity = model.n_similarity (text1, text2)
print ("Similarity Index: {:4.2f} %".format(similarity*100))

Similarity Index: 80.04 %


D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.
